# Formative Assignment: Advanced Linear Algebra (PCA)
This notebook will guide you through the implementation of Principal Component Analysis (PCA). Fill in the missing code and provide the required answers in the appropriate sections. You will work with a dataset that is Africanized.

**Student Name:** Completed Example

**Dataset:** African Agriculture and Food Security Indicators

Make sure to display the output for each code cell after running it.

### Step 0: Import Libraries and Load African Dataset
First, we'll import necessary libraries and create our Africanized dataset.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

In [ ]:
# Create African Agriculture and Food Security Dataset
# This dataset contains agricultural and food security indicators for 45 African countries

countries = [
    'Nigeria', 'Ethiopia', 'Egypt', 'Democratic Republic of Congo', 'Tanzania',
    'South Africa', 'Kenya', 'Uganda', 'Algeria', 'Sudan',
    'Morocco', 'Angola', 'Ghana', 'Mozambique', 'Madagascar',
    'Cameroon', "Côte d'Ivoire", 'Niger', 'Burkina Faso', 'Mali',
    'Malawi', 'Zambia', 'Somalia', 'Senegal', 'Chad',
    'Zimbabwe', 'Guinea', 'Rwanda', 'Benin', 'Burundi',
    'Tunisia', 'Togo', 'Sierra Leone', 'Libya', 'Liberia',
    'Mauritania', 'Central African Republic', 'Eritrea', 'Lesotho', 'Namibia',
    'Botswana', 'Gabon', 'Gambia', 'Mauritius', 'Eswatini'
]

climate_zones = ['Arid', 'Semi-Arid', 'Tropical', 'Sub-Tropical', 'Mediterranean']
n = len(countries)

# Generate realistic correlated agricultural data
# Base productivity that will influence other variables
base_productivity = np.random.gamma(5, 2, n)

data = {
    'Country': countries,
    'Climate_Zone': np.random.choice(climate_zones, n),
    'Cereal_Production_MT': base_productivity * np.random.lognormal(2, 0.5, n),
    'Arable_Land_Percent': np.random.beta(3, 5, n) * 100,
    'Agricultural_Employment_Percent': np.random.beta(6, 2, n) * 100,
    'Fertilizer_Use_KG_Per_Hectare': np.random.gamma(3, 15, n),
    'Irrigation_Percent': np.random.beta(2, 8, n) * 100,
    'Crop_Yield_Index': base_productivity * np.random.normal(100, 20, n),
    'Food_Production_Index': base_productivity * np.random.normal(110, 25, n),
    'Livestock_Production_Index': np.random.gamma(8, 12, n),
    'Undernourishment_Percent': np.random.beta(3, 4, n) * 50,
    'Agricultural_Value_Added_Percent_GDP': np.random.beta(4, 2, n) * 60,
    'Rural_Population_Percent': np.random.beta(5, 3, n) * 100,
    'Access_To_Electricity_Rural_Percent': np.random.beta(3, 5, n) * 100,
    'Agricultural_Machinery_Per_100Sq_KM': np.random.gamma(2, 8, n),
    'Food_Import_Dependency_Percent': np.random.beta(3, 4, n) * 80
}

df = pd.DataFrame(data)

# Introduce missing values (REQUIREMENT: dataset must have missing values)
missing_columns = ['Fertilizer_Use_KG_Per_Hectare', 'Irrigation_Percent', 'Undernourishment_Percent',
                   'Access_To_Electricity_Rural_Percent', 'Food_Import_Dependency_Percent']

for col in missing_columns:
    missing_indices = np.random.choice(df.index, size=int(0.18 * len(df)), replace=False)
    df.loc[missing_indices, col] = np.nan

print("African Agriculture and Food Security Dataset Created!")
print(f"\nDataset Shape: {df.shape}")
print(f"\nFirst 5 rows:")
df.head()

In [ ]:
# Display full dataset overview
print("Complete Dataset Overview:")
print("=" * 70)
df

In [ ]:
# Check for missing values (REQUIRED)
print("Missing Values Summary:")
print("=" * 50)
missing_summary = df.isnull().sum()
print(missing_summary[missing_summary > 0])
print(f"\nTotal missing values: {df.isnull().sum().sum()}")

# Check data types (REQUIRED: at least 1 non-numeric column)
print("\n" + "=" * 50)
print("Data Types:")
print("=" * 50)
print(df.dtypes)
print(f"\nNon-numeric columns: {df.select_dtypes(include=['object']).columns.tolist()}")

In [ ]:
# Preprocess: Handle missing values and separate numeric data
print("Data Preprocessing...")
print("=" * 50)

# Identify categorical and numeric columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

print(f"Categorical columns: {categorical_cols}")
print(f"Numeric columns: {numeric_cols}")
print(f"\nNumber of numeric features: {len(numeric_cols)}")

# Create a copy with only numeric columns for PCA
df_numeric = df[numeric_cols].copy()

# Handle missing values using mean imputation
print("\nHandling missing values using mean imputation...")
for col in df_numeric.columns:
    if df_numeric[col].isnull().any():
        mean_value = df_numeric[col].mean()
        df_numeric[col].fillna(mean_value, inplace=True)
        print(f"  - Filled {col} with mean: {mean_value:.2f}")

# Verify no missing values remain
print(f"\nMissing values after imputation: {df_numeric.isnull().sum().sum()}")

# Convert to numpy array for PCA implementation
data_array = df_numeric.values

print(f"\nData array shape: {data_array.shape}")
print("Preprocessing complete!")

### Step 1: Load and Standardize the Data
Before applying PCA, we must standardize the dataset. Standardization ensures that all features have a mean of 0 and a standard deviation of 1, which is essential for PCA.
Fill in the code to standardize the dataset.

STRICTLY - Write code that implements standardization based on the formula:

**Standardized Data = (Data - Mean) / Standard Deviation**

In [ ]:
# Step 1: Load and Standardize the data (use of numpy only allowed)
# Formula: standardized_data = (data - mean) / std

# Calculate mean and standard deviation
mean = np.mean(data_array, axis=0)
std = np.std(data_array, axis=0)

# Standardize the data: (Data - Mean) / Standard Deviation
standardized_data = (data_array - mean) / std

print("Data Standardization Complete!")
print("=" * 70)
print(f"Original data shape: {data_array.shape}")
print(f"Standardized data shape: {standardized_data.shape}")
print(f"\nStandardized data mean (should be ~0): {np.mean(standardized_data, axis=0)[:5]}")
print(f"Standardized data std (should be ~1): {np.std(standardized_data, axis=0)[:5]}")

### Step 2: Display Sample of Standardized Data
Let's visualize what standardization did to our data.

In [ ]:
# Create a DataFrame to display standardized data
standardized_df = pd.DataFrame(standardized_data, columns=df_numeric.columns)
print("Sample of Standardized Data:")
print("=" * 70)
standardized_df.head(10)

### Step 3: Calculate the Covariance Matrix
The covariance matrix helps us understand how the features are related to each other. It is a key component in PCA.

**Formula: Covariance Matrix = (1/(n-1)) × X^T × X**

where X is the standardized data matrix.

In [ ]:
# Step 3: Calculate the Covariance Matrix
# Formula: cov_matrix = (1/(n-1)) * X^T * X

# Get number of samples
n_samples = standardized_data.shape[0]

# Calculate covariance matrix using numpy
cov_matrix = np.dot(standardized_data.T, standardized_data) / (n_samples - 1)

print("Covariance Matrix Calculated!")
print("=" * 70)
print(f"Covariance matrix shape: {cov_matrix.shape}")
print(f"\nFirst 5x5 section of covariance matrix:")
print(cov_matrix[:5, :5])

In [ ]:
# Visualize the covariance matrix
plt.figure(figsize=(14, 12))
sns.heatmap(cov_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1,
            xticklabels=df_numeric.columns,
            yticklabels=df_numeric.columns)
plt.title('Covariance Matrix Heatmap - African Agriculture Dataset', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Positive values (red): Features move together in the same direction")
print("- Negative values (blue): Features move in opposite directions")
print("- Diagonal values: Variance of each feature (should be ~1 for standardized data)")

### Step 4: Perform Eigendecomposition
Eigendecomposition of the covariance matrix will give us the eigenvalues and eigenvectors, which are essential for PCA.
Fill in the code to compute the eigenvalues and eigenvectors of the covariance matrix.

**Formula: C × v = λ × v**

where:
- C is the covariance matrix
- λ (lambda) are the eigenvalues
- v are the eigenvectors

In [ ]:
# Step 4: Perform Eigendecomposition
# Use numpy's linalg.eig function to compute eigenvalues and eigenvectors

eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

# Convert to real values (remove any imaginary components due to numerical precision)
eigenvalues = eigenvalues.real
eigenvectors = eigenvectors.real

print("Eigendecomposition Complete!")
print("=" * 70)
print(f"Number of eigenvalues: {len(eigenvalues)}")
print(f"Eigenvectors shape: {eigenvectors.shape}")
print(f"\nFirst 5 eigenvalues: {eigenvalues[:5]}")

### Step 5: Sort Principal Components
Sort the eigenvectors based on their corresponding eigenvalues in descending order. The higher the eigenvalue, the more important the eigenvector.
Complete the code to sort the eigenvectors and print the sorted components.

**Why sort?** The eigenvalue represents the amount of variance explained by its corresponding eigenvector (principal component).

In [ ]:
# Step 5: Sort Principal Components
# Sort eigenvalues in descending order and reorder eigenvectors accordingly

# Get indices that would sort eigenvalues in descending order
sorted_indices = np.argsort(eigenvalues)[::-1]

# Sort eigenvalues in descending order
sorted_eigenvalues = eigenvalues[sorted_indices]

# Reorder eigenvectors based on sorted eigenvalues
sorted_eigenvectors = eigenvectors[:, sorted_indices]

print("Principal Components Sorted!")
print("=" * 70)
print("Sorted eigenvalues (variance explained by each component):")
for i, eigenvalue in enumerate(sorted_eigenvalues):
    print(f"PC{i+1}: {eigenvalue:.4f}")

### Step 6: Calculate Explained Variance
Calculate the proportion of variance explained by each principal component and the cumulative variance.

In [ ]:
# Step 6: Calculate Explained Variance
# Calculate explained variance ratio
total_variance = np.sum(sorted_eigenvalues)
explained_variance_ratio = sorted_eigenvalues / total_variance
cumulative_variance = np.cumsum(explained_variance_ratio)

# Create a DataFrame for better visualization
variance_df = pd.DataFrame({
    'Principal Component': [f'PC{i+1}' for i in range(len(sorted_eigenvalues))],
    'Eigenvalue': sorted_eigenvalues,
    'Variance Explained (%)': explained_variance_ratio * 100,
    'Cumulative Variance (%)': cumulative_variance * 100
})

print("Explained Variance by Principal Components:")
print("=" * 70)
variance_df

In [ ]:
# Visualize explained variance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Scree plot
ax1.bar(range(1, len(sorted_eigenvalues) + 1), explained_variance_ratio * 100, 
        alpha=0.7, color='steelblue', edgecolor='black')
ax1.set_xlabel('Principal Component', fontsize=12, fontweight='bold')
ax1.set_ylabel('Variance Explained (%)', fontsize=12, fontweight='bold')
ax1.set_title('Scree Plot - Variance Explained by Each PC', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

# Cumulative variance plot
ax2.plot(range(1, len(sorted_eigenvalues) + 1), cumulative_variance * 100, 
         marker='o', linewidth=2, markersize=8, color='darkgreen')
ax2.axhline(y=80, color='red', linestyle='--', linewidth=2, label='80% Threshold')
ax2.axhline(y=90, color='orange', linestyle='--', linewidth=2, label='90% Threshold')
ax2.set_xlabel('Number of Principal Components', fontsize=12, fontweight='bold')
ax2.set_ylabel('Cumulative Variance Explained (%)', fontsize=12, fontweight='bold')
ax2.set_title('Cumulative Variance Explained', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Find number of components for 80% and 90% variance
n_components_80 = np.argmax(cumulative_variance >= 0.80) + 1
n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1

print(f"\nNumber of components needed for 80% variance: {n_components_80}")
print(f"Number of components needed for 90% variance: {n_components_90}")

## Task 1: Project Data onto Principal Components
Now that we have the principal components, project the standardized data onto them to get the transformed data.

In [ ]:
# Task 1: Project data onto principal components
# Select number of components (let's use enough for 90% variance)
n_components = n_components_90

# Select the top n_components eigenvectors
principal_components = sorted_eigenvectors[:, :n_components]

# Project the data: Transformed Data = Standardized Data × Principal Components
transformed_data = np.dot(standardized_data, principal_components)

print(f"Data Projection Complete!")
print("=" * 70)
print(f"Original data shape: {standardized_data.shape}")
print(f"Transformed data shape: {transformed_data.shape}")
print(f"Dimensionality reduction: {standardized_data.shape[1]} → {transformed_data.shape[1]} features")
print(f"Variance retained: {cumulative_variance[n_components-1]*100:.2f}%")

In [ ]:
# Create DataFrame with transformed data
transformed_df = pd.DataFrame(
    transformed_data,
    columns=[f'PC{i+1}' for i in range(n_components)]
)
transformed_df['Country'] = df['Country'].values

print("Transformed Data (First 10 countries):")
print("=" * 70)
transformed_df.head(10)

In [ ]:
# Visualize data in PC space (2D projection)
plt.figure(figsize=(12, 8))
scatter = plt.scatter(transformed_data[:, 0], transformed_data[:, 1], 
                     c=range(len(countries)), cmap='viridis', 
                     s=100, alpha=0.6, edgecolors='black', linewidth=1)

# Annotate some countries
for i, country in enumerate(countries[::5]):  # Label every 5th country to avoid clutter
    plt.annotate(country, (transformed_data[i*5, 0], transformed_data[i*5, 1]),
                fontsize=8, alpha=0.7)

plt.xlabel(f'PC1 ({explained_variance_ratio[0]*100:.1f}% variance)', fontsize=12, fontweight='bold')
plt.ylabel(f'PC2 ({explained_variance_ratio[1]*100:.1f}% variance)', fontsize=12, fontweight='bold')
plt.title('African Countries in Principal Component Space', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='Country Index')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### Step 7: Analyze Component Loadings
Understand which original features contribute most to each principal component.

In [ ]:
# Create loadings matrix (correlation between original features and PCs)
loadings = sorted_eigenvectors[:, :n_components] * np.sqrt(sorted_eigenvalues[:n_components])

# Create DataFrame for better visualization
loadings_df = pd.DataFrame(
    loadings,
    columns=[f'PC{i+1}' for i in range(n_components)],
    index=df_numeric.columns
)

print("Component Loadings (Feature Contributions):")
print("=" * 70)
loadings_df

In [ ]:
# Visualize loadings for first 3 PCs
fig, axes = plt.subplots(1, min(3, n_components), figsize=(18, 6))
if n_components == 1:
    axes = [axes]

for i in range(min(3, n_components)):
    ax = axes[i]
    loadings_sorted = loadings_df[f'PC{i+1}'].sort_values()
    colors = ['red' if x < 0 else 'green' for x in loadings_sorted]
    loadings_sorted.plot(kind='barh', ax=ax, color=colors, alpha=0.7, edgecolor='black')
    ax.set_xlabel('Loading Value', fontsize=11, fontweight='bold')
    ax.set_title(f'PC{i+1} Loadings ({explained_variance_ratio[i]*100:.1f}% variance)', 
                fontsize=12, fontweight='bold')
    ax.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
    ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Positive loadings (green): Feature increases with PC")
print("- Negative loadings (red): Feature decreases with PC")
print("- Larger absolute values: Stronger contribution to the PC")

## Task 2: Dynamic Component Selection Based on Variance Threshold
Implement a function that dynamically selects the number of components based on a desired variance threshold.

In [ ]:
# Task 2: Dynamic component selection
def select_components_by_variance(eigenvalues, variance_threshold=0.95):
    """
    Select the minimum number of components needed to explain
    at least variance_threshold of the total variance.
    
    Parameters:
    -----------
    eigenvalues : array
        Sorted eigenvalues in descending order
    variance_threshold : float
        Desired proportion of variance to retain (default: 0.95)
    
    Returns:
    --------
    n_components : int
        Number of components needed
    """
    # Calculate total variance
    total_var = np.sum(eigenvalues)
    
    # Calculate explained variance ratio
    var_ratio = eigenvalues / total_var
    
    # Calculate cumulative variance
    cumulative_var = np.cumsum(var_ratio)
    
    # Find the number of components needed
    n_components = np.argmax(cumulative_var >= variance_threshold) + 1
    
    return n_components

# Test the function with different thresholds
print("Dynamic Component Selection:")
print("=" * 70)
for threshold in [0.80, 0.85, 0.90, 0.95, 0.99]:
    n = select_components_by_variance(sorted_eigenvalues, threshold)
    print(f"Variance threshold {threshold*100:.0f}%: {n} components needed")

## Task 3: Performance Optimization and Reconstruction
Implement data reconstruction from principal components and calculate reconstruction error.

In [ ]:
# Task 3: Data reconstruction and error calculation
def reconstruct_data(transformed_data, principal_components, mean, std):
    """
    Reconstruct original data from principal components.
    
    Parameters:
    -----------
    transformed_data : array
        Data in PC space
    principal_components : array
        Eigenvectors used for transformation
    mean : array
        Original data mean (for un-standardizing)
    std : array
        Original data standard deviation (for un-standardizing)
    
    Returns:
    --------
    reconstructed_data : array
        Reconstructed original data
    """
    # Inverse transform: PC Space → Standardized Space
    standardized_reconstructed = np.dot(transformed_data, principal_components.T)
    
    # Un-standardize: Standardized → Original Scale
    reconstructed_data = (standardized_reconstructed * std) + mean
    
    return reconstructed_data

# Reconstruct data using different numbers of components
print("Data Reconstruction Analysis:")
print("=" * 70)

for n in [2, 5, n_components_90, len(sorted_eigenvalues)]:
    # Project data
    pcs = sorted_eigenvectors[:, :n]
    projected = np.dot(standardized_data, pcs)
    
    # Reconstruct
    reconstructed = reconstruct_data(projected, pcs, mean, std)
    
    # Calculate reconstruction error (Mean Squared Error)
    mse = np.mean((data_array - reconstructed) ** 2)
    rmse = np.sqrt(mse)
    
    # Calculate R² score
    ss_tot = np.sum((data_array - np.mean(data_array, axis=0)) ** 2)
    ss_res = np.sum((data_array - reconstructed) ** 2)
    r2_score = 1 - (ss_res / ss_tot)
    
    print(f"\nComponents: {n}")
    print(f"  Variance explained: {cumulative_variance[n-1]*100:.2f}%")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R² Score: {r2_score:.4f}")

In [ ]:
# Visualize reconstruction quality
n_test = 5  # Use 5 components for visualization
pcs_test = sorted_eigenvectors[:, :n_test]
projected_test = np.dot(standardized_data, pcs_test)
reconstructed_test = reconstruct_data(projected_test, pcs_test, mean, std)

# Compare original vs reconstructed for first 3 features
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i in range(3):
    ax = axes[i]
    ax.scatter(data_array[:, i], reconstructed_test[:, i], alpha=0.6, s=50)
    ax.plot([data_array[:, i].min(), data_array[:, i].max()],
           [data_array[:, i].min(), data_array[:, i].max()],
           'r--', linewidth=2, label='Perfect Reconstruction')
    ax.set_xlabel(f'Original {df_numeric.columns[i]}', fontsize=11, fontweight='bold')
    ax.set_ylabel(f'Reconstructed {df_numeric.columns[i]}', fontsize=11, fontweight='bold')
    ax.set_title(f'{df_numeric.columns[i]}', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)

plt.suptitle(f'Original vs Reconstructed Data ({n_test} Components)', 
            fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Final Analysis: Interpretation of Results

In [ ]:
print("=" * 70)
print("FINAL PCA ANALYSIS SUMMARY")
print("=" * 70)
print(f"\nDataset: African Agriculture and Food Security Indicators")
print(f"Countries analyzed: {len(countries)}")
print(f"Original features: {len(df_numeric.columns)}")
print(f"\nDimensionality Reduction:")
print(f"  - {n_components_80} components explain 80% of variance")
print(f"  - {n_components_90} components explain 90% of variance")
print(f"  - {len(sorted_eigenvalues)} components explain 100% of variance")
print(f"\nTop 3 Principal Components:")
for i in range(min(3, len(sorted_eigenvalues))):
    print(f"  PC{i+1}: {explained_variance_ratio[i]*100:.2f}% variance")
    top_features = loadings_df[f'PC{i+1}'].abs().nlargest(3)
    print(f"       Top features: {', '.join(top_features.index.tolist())}")

print(f"\nKey Insights:")
print(f"  - PCA successfully reduced dimensionality from {len(df_numeric.columns)} to {n_components_90} features")
print(f"  - Retained {cumulative_variance[n_components_90-1]*100:.1f}% of original information")
print(f"  - Most variance captured by first few components (as expected)")
print("=" * 70)

## Bonus: 3D Visualization of Principal Components

In [ ]:
# 3D visualization of first 3 principal components
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(transformed_data[:, 0], 
                    transformed_data[:, 1], 
                    transformed_data[:, 2],
                    c=range(len(countries)), 
                    cmap='viridis',
                    s=100, 
                    alpha=0.6, 
                    edgecolors='black',
                    linewidth=1)

# Label some countries
for i in range(0, len(countries), 7):  # Label every 7th country
    ax.text(transformed_data[i, 0], 
           transformed_data[i, 1], 
           transformed_data[i, 2],
           countries[i], 
           fontsize=8)

ax.set_xlabel(f'PC1 ({explained_variance_ratio[0]*100:.1f}%)', fontsize=11, fontweight='bold')
ax.set_ylabel(f'PC2 ({explained_variance_ratio[1]*100:.1f}%)', fontsize=11, fontweight='bold')
ax.set_zlabel(f'PC3 ({explained_variance_ratio[2]*100:.1f}%)', fontsize=11, fontweight='bold')
ax.set_title('3D Principal Component Space - African Agriculture', fontsize=14, fontweight='bold')

plt.colorbar(scatter, ax=ax, label='Country Index', shrink=0.8)
plt.tight_layout()
plt.show()

## Conclusion

In this assignment, you have successfully:

1. ✅ Loaded and preprocessed an Africanized dataset with missing values and non-numeric columns
2. ✅ Implemented data standardization from scratch using numpy
3. ✅ Calculated the covariance matrix manually
4. ✅ Performed eigendecomposition to extract principal components
5. ✅ Sorted components by explained variance
6. ✅ Projected data onto principal component space
7. ✅ Implemented dynamic component selection based on variance threshold
8. ✅ Reconstructed data and analyzed reconstruction error
9. ✅ Visualized results using multiple plotting techniques

**Key Takeaways:**
- PCA is a powerful dimensionality reduction technique
- Standardization is crucial before applying PCA
- First few components typically capture most variance
- Component loadings help interpret what each PC represents
- Trade-off between dimensionality reduction and information loss